In [40]:
# CARGAMOS LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

import boto3
import s3fs
import fastparquet
import awswrangler as wr
import os

import pyarrow.parquet as pq
import pyarrow as pa

import requests
import json
from io import BytesIO
from pandas.tseries.offsets import BDay

import math

In [42]:
# INICIALIZAMOS UNA SESION EN AWS
session = boto3.Session(
    aws_access_key_id=os.environ['S3_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_SECRET_KEY'],
    region_name=os.environ['S3_REGION'])

s3_client = boto3.client('s3')

OBJETIVO DE ESTE NOTEBOOK:
- DESCARGAR DE DISTINTOS ORIGENES LOS DATOS DE LOS ACTIVOS DEL IBEX35
- COMPARAR LOS DATOS OBTENIDOS 

## CARGA DE COMPOSICIÓN HISTÓRICA DE IBEX35

In [43]:
# LEEMOS EL FICHERO CON LA COMPOSICIÓN HISTORICA
ibex_constituents = pd.read_excel('data_processing/Historical Composition IBEX35.xlsx')
ibex_constituents.head()

,Activo,Inclusion,Exclusion
0,ACE,1991-01-02,2003-06-02
1,ACX,1991-01-02,1991-07-01
2,ALB,1991-01-02,1991-07-01
3,ASL,1991-01-02,1994-01-03
4,BBV,1991-01-02,2000-01-31


In [44]:
# ibex_constituents['Exclusion'] -= BDay(1)

# ibex_constituents['Inclusion'] = ibex_constituents['Inclusion'].dt.date
# ibex_constituents['Exclusion'] = ibex_constituents['Exclusion'].dt.date

# OBTENEMOS EL DIA ACTUAL
today = datetime.date.today().strftime('%Y-%m-%d')

# COMPLETAMOS LOS DATOS NULOS EN EL FICHERO DE COMPOSICIÓN HISTORICA CON LA FECHA ACTUAL
ibex_constituents.fillna(today,inplace=True)

# DEFINIMOS UNA VARIABLE PARA TODOS LOS ACTIVOS
activos = ibex_constituents.Activo
ibex_constituents['Ticker'] = activos.apply(lambda x: x.split('_')[0])

unique_activos = list(set(activos))

## DESCARGA DE DATOS CON API DE MARKETSTACK 

In [45]:
# OBTENEMOS TODOS LOS EXCHANGES
url = f'http://api.marketstack.com/v1/'
exchange_list = 'exchanges'

r = requests.get(url+exchange_list,params={
    'access_key':API_TOKEN
    })
# print(r.content)
exchanges = pd.DataFrame(json.load(BytesIO(r.content))['data'])
exchanges


,name,acronym,mic,country,country_code,city,website,timezone,currency
0,NASDAQ Stock Exchange,NASDAQ,XNAS,USA,US,New York,www.nasdaq.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."
1,New York Stock Exchange,NYSE,XNYS,USA,US,New York,www.nyse.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."
2,NYSE ARCA,NYSEARCA,ARCX,USA,US,New York,www.nyse.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."
3,OTC Markets,,OTCM,USA,US,New York,www.otcmarkets.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."
4,Buenos Aires Stock Exchange,BCBA,XBUE,Argentina,AR,Buenos Aires,www.bcba.sba.com.ar,"{'timezone': 'America/Argentina/Buenos_Aires',...","{'code': 'ARS', 'symbol': 'AR$', 'name': 'Arge..."
...,...,...,...,...,...,...,...,...,...
64,OTCQB Marketplace,OTCQB,OTCB,USA,US,New York,www.otcmarkets.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."
65,OTCQX Marketplace,OTCQX,OTCQ,USA,US,New York,www.otcmarkets.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."
66,OTC PINK current,PINK,PINC,USA,US,New York,www.otcmarkets.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."
67,Investors Exchange,IEX,IEXG,USA,US,New York,www.iextrading.com,"{'timezone': 'America/New_York', 'abbr': 'EST'...","{'code': 'USD', 'symbol': '$', 'name': 'US Dol..."


In [46]:
spain_ex_code = exchanges.loc[exchanges['country']=='Spain','mic'].values[0]
spain_ex_code

'BMEX'

In [47]:
# OBTENEMOS UNA LISTA DE LOS ACTIVOS DISPONIBLES EN MARKETSTACK
spain_ex_code = exchanges.loc[exchanges['country']=='Spain','mic'].values[0]

symbol_list = 'tickers'

r = requests.get(url+symbol_list,
                 params={
                    'access_key': API_TOKEN,
                    'exchange': spain_ex_code,
                    'limit':1000
                    }
                )
# print(r.content)
symbols = pd.DataFrame(json.load(BytesIO(r.content))['data'])
symbols

,name,symbol,has_intraday,has_eod,country,stock_exchange
0,INDUSTRIA DE DISEO TEXTIL S.A. INDITEX-,ITX.BMEX,False,True,None,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
1,AIRBUS,AIR.BMEX,False,True,None,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
2,"PETROLEO BRASILEIRO,S.A. -PETROBRAS- ORDINARIAS",XPBR.BMEX,False,False,None,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
3,ACCIONES IBERDROLA,IBE.BMEX,False,True,None,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
4,BANCO SANTANDER S.A.,SAN.BMEX,False,True,None,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
...,...,...,...,...,...,...
304,Silicius Real Estate SOCIMI S.A.,YSIL.BMEX,False,True,Spain,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
305,Tempore Properties SOCIMI SAU,YTEM.BMEX,False,True,Spain,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
306,Urban View Development Spain SOCIMI S.A.,YUVS.BMEX,False,True,Spain,"{'name': 'Bolsas y Mercados Españoles', 'acron..."
307,Veracruz Properties SOCIMI SA,YVCP.BMEX,False,True,Spain,"{'name': 'Bolsas y Mercados Españoles', 'acron..."


In [49]:
ibex_constituents[ibex_constituents['Activo'].isin(['SCYR','SCYR_1','SCYR_2'])].index

Int64Index([103, 109, 141], dtype='int64')

In [50]:
# REALIZAMOS LA DESCARGA DE LOS DATOS DE MARKETSTACK Y LOS CARGAMOS EN UN BUCKET DE S3
bucket_name = 'stock-market-historical-data'
raw_folder = 'preprocessed/marketstack/'
s3 = boto3.resource('s3')
failed_loads = []
eod_data = 'eod'
indexes = ibex_constituents[ibex_constituents['Activo'].isin(['SCYR','SCYR_1','SCYR_2'])].index

# for i in range(ibex_constituents.shape[0]):
for i in indexes:

    try:
        ticker = ibex_constituents.Ticker[i] + '.BMEX'
        file_name = ibex_constituents.Activo[i] +'.parquet'
        start_date = ibex_constituents.Inclusion[i]
        end_date = ibex_constituents.Exclusion[i]
        full_path = raw_folder + file_name

        total_days = (end_date.to_period('D') - start_date.to_period('D')).n/1000

        stock_data = pd.DataFrame()

        for _ in range(math.ceil(total_days)):
            try:
                prov_end_date = start_date + datetime.timedelta(days=1000)
                if prov_end_date > end_date:
                    r = requests.get(url+eod_data,
                            params={
                                'access_key': API_TOKEN,
                                'exchange': spain_ex_code,
                                'symbols':ticker,
                                'date_from': start_date,
                                'date_to': end_date,
                                'limit':1000
                                }
                            )
                else:
                    r = requests.get(url+eod_data,
                            params={
                                'access_key': API_TOKEN,
                                'exchange': spain_ex_code,
                                'symbols':ticker,
                                'date_from': start_date,
                                'date_to': prov_end_date,
                                'limit': 1000
                                }
                            )
                

                ticker_data = pd.DataFrame(json.load(BytesIO(r.content))['data'])
                ticker_data.set_index('date', inplace=True)

                stock_data = pd.concat([stock_data, ticker_data], axis=0)
                start_date = prov_end_date + datetime.timedelta(days=1)
            except:
                continue
        table = pa.Table.from_pandas(stock_data)
        buf = BytesIO()
        pq.write_table(table, buf)
        
        s3.Object(bucket_name, full_path).put(Body=buf.getvalue())
        print(ibex_constituents.Activo[i])
    except:
        print('failed')
        failed_loads.append(ticker)
        continue

SCYR
SCYR_1
SCYR_2


In [38]:
r = requests.get(url+eod_data,
                            params={
                                'access_key': API_TOKEN,
                                'exchange': 'BMEX',
                                'symbols':'MVC.BMEX',
                                # 'from_date':'1991-01-02',
                                # 'to_date':'1993-01-04',
                                'limit':1000,
                                'offset': 642
                                }
                            )

r.content

b'{"pagination":{"limit":1000,"offset":642,"count":0,"total":642},"data":[]}'

In [51]:
raw_folder = 'preprocessed/marketstack/'
raw_path = bucket_name + raw_folder
objects_dict = s3_client.list_objects_v2(
        Bucket=bucket_name,
        Prefix ='preprocessed/marketstack')
raw_filepaths = [item['Key'] for item in objects_dict['Contents'] if item['Key'].endswith('.parquet')]
act_list = [i.split('/')[-1].replace('.parquet','') for i in raw_filepaths]

In [52]:
ibex_historical_data = {}
processed_folder = 'processed/marketstack/'
file_name = 'ibex_historical_data'
full_path = processed_folder + file_name

In [53]:
def download_s3_parquet_file(s3, bucket, key):
    buffer = BytesIO()
    s3.Object(bucket, key).download_fileobj(buffer)
    return buffer

In [54]:
# CARGAMOS EL DATAFRAME CON TODOS LOS DATOS DE CIERRE DE TODOS LOS ACTIVOS DESDE 02/01/1991
for i in range(len(raw_filepaths)):

    try:
        close_price = pq.read_table(download_s3_parquet_file(s3, bucket_name,raw_filepaths[i])).to_pandas()['adj_close']
        close_price.index = pd.to_datetime(close_price.index).strftime('%Y-%m-%d')
        ibex_historical_data[act_list[i]] = close_price
    except:
        ibex_historical_data[act_list[i]] = np.nan
        

ibex_historical_data = pd.DataFrame(ibex_historical_data)

table = pa.Table.from_pandas(ibex_historical_data)
buf = BytesIO()
pq.write_table(table, buf)

s3.Object(bucket_name, full_path).put(Body=buf.getvalue())

{'ResponseMetadata': {'RequestId': 'N2A8Y7E16A3E2F19',
  'HostId': '21yKgG31c1XkzAhiIrkJdrfoauWidwmd55EJkUuG4Q8GrlBjJzzZPW9puk8AnObeyk7QvPTBbOo=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '21yKgG31c1XkzAhiIrkJdrfoauWidwmd55EJkUuG4Q8GrlBjJzzZPW9puk8AnObeyk7QvPTBbOo=',
   'x-amz-request-id': 'N2A8Y7E16A3E2F19',
   'date': 'Fri, 11 Aug 2023 08:36:08 GMT',
   'x-amz-version-id': 'mAoC2OricBQHLd4Y4EkgNQ_mcPJhcOCo',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"10f6ece86af2ab6599c5b3df05f3e177"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"10f6ece86af2ab6599c5b3df05f3e177"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'mAoC2OricBQHLd4Y4EkgNQ_mcPJhcOCo'}

In [41]:
ibex_historical_data

,A3TV,ABE,ABG.P,ABG.P_1,ABG,ACE,ACR,ACS,ACX,ACX_1,...,URB,VAL,VAL_1,VDR,VIS,VIS_1,VIS_2,VIS_3,ZEL,ZOT
date,,,,,,,,,,,,,,,,,,,,,
1993-08-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0929,...,NaN,NaN,NaN,NaN,2.8846,NaN,NaN,NaN,NaN,0.4108
1993-08-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0929,...,NaN,NaN,NaN,NaN,2.8678,NaN,NaN,NaN,NaN,0.4129
1993-08-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1355,...,NaN,NaN,NaN,NaN,2.8846,NaN,NaN,NaN,NaN,0.4204
1993-08-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.1817,...,NaN,NaN,NaN,NaN,2.9517,NaN,NaN,NaN,NaN,0.4146
1993-08-13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.2025,...,NaN,NaN,NaN,NaN,2.9602,NaN,NaN,NaN,NaN,0.4204
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-08-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.15,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.36,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-08-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.44,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [42]:
failed_loads = ibex_historical_data.columns[ibex_historical_data.isnull().values.all(axis=0)]
failed_loads, len(failed_loads)

(Index(['A3TV', 'ABE', 'ABG.P', 'ABG.P_1', 'ACE', 'ACR', 'ACX', 'AGR', 'AGS',
        'AGS_1', 'AGS_2', 'AGS_3', 'ALB', 'ALB_1', 'ALT', 'AMS', 'ARA', 'ARG',
        'ASL', 'AUM', 'AZC', 'AZC_1', 'BBV', 'BCH', 'BTO', 'BTO_1', 'BTO_2',
        'CAN', 'CAR', 'CEN', 'CEP', 'CEP_1', 'CIN', 'CRF', 'CRI', 'CRI_1',
        'CRI_2', 'CTE', 'CTF', 'CTG', 'CUB', 'DRC', 'EBA', 'ECR', 'ELE',
        'ELE_1', 'EVA', 'EXT', 'FAD', 'FEC', 'FER', 'FOC', 'GAM', 'GAM_1',
        'GAS', 'GES', 'GES_1', 'GPP', 'HHU', 'HHU_1', 'HID', 'HIS', 'IBLA',
        'IBR', 'JAZ', 'LOR', 'MVC', 'MVC_1', 'OHLA', 'PMD', 'POP', 'PRY', 'PUL',
        'RAD', 'SAR', 'SCH', 'SEV', 'SGC', 'SOL', 'SYV', 'SYV_1', 'TAB', 'TEM',
        'TPI', 'TPZ', 'TRR', 'TRR_1', 'UNF', 'UNI', 'URA', 'URB', 'VAL',
        'VAL_1', 'VDR', 'ZEL'],
       dtype='object'),
 95)